In [84]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("postoffice.csv")

# Define the count value
count = 100  # Change this value as needed

columns_to_remove = ['duration', 'orig_name', 'orig_county', 'continuous', 'alt_name', 'discontinued', 'county1', 'county2', 'county3',  'id', 'gnis_name', 'gnis_match', 'gnis_county', 'gnis_state', 'gnis_feature_id', 'gnis_feature_class', 'gnis_orig_name', 'gnis_orig_county', 'gnis_latitude', 'gnis_longitude', 'gnis_elev_in_m', 'gnis_dist']

# Remove the specified columns
df_cleaned = df.drop(columns=columns_to_remove)
df_cleaned = df_cleaned.dropna(subset=['latitude', 'longitude', 'established'])

# Filter the DataFrame based on the condition
df_cleaned = df_cleaned[df_cleaned['established'].apply(lambda x: 2021 - x < count)]

print(df_cleaned)


               name state  established  stamp_index  coordinates   latitude  \
14           ABAUGH    AR       1928.0          1.0         True  35.781747   
163          ACTION    AL       1924.0          5.0         True  33.426775   
220           ADAMS    MS       1950.0          1.0         True  32.159874   
295        ADELBERT    MO       1922.0          4.0         True  37.970883   
467       ALABASTER    AL       1951.0          0.0         True  33.217264   
...             ...   ...          ...          ...          ...        ...   
166010       ZAMORA    NM       1938.0          2.0         True  35.079570   
166013         ZANE    UT       1925.0          5.0         True  37.925250   
166022     ZARAGOZA    TX       1925.0          2.0         True  26.680060   
166056       ZENIFF    AZ       1922.0          3.0         True  34.577260   
166072  ZEPHYR COVE    NV       1930.0          0.0         True  38.985540   

         longitude  
14      -93.501574  
163     -

In [85]:

# Convert 'duration' and 'established' columns to integers
#df_top_1000_recent['duration'] = df_top_1000_recent['duration'].astype(int)
df_cleaned['established'] = df_cleaned['established'].astype(int)


# Write the cleaned data to a new CSV file
df_cleaned.to_csv("postoffice_cleaned.csv", index=False)

print("Rows with NA values removed. Cleaned data saved to postoffice_cleaned.csv")


Rows with NA values removed. Cleaned data saved to postoffice_cleaned.csv


In [86]:
import pandas as pd

# Read the cleaned CSV file
df_cleaned = pd.read_csv("postoffice_cleaned.csv")

# Dictionary mapping short names to long names of states
state_names_long = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

# Invert the dictionary to get short names as keys and long names as values
state_names_short = {v: k for k, v in state_names_long.items()}

# Map short state names to long state names
df_cleaned['state_longform'] = df_cleaned['state'].map(state_names_long)

# Aggregate number of post offices per state
state_post_offices_long = df_cleaned.groupby('state_longform').size().reset_index(name='num_post_offices')

# Map long state names back to short state names
state_post_offices_long['state'] = state_post_offices_long['state_longform'].map(state_names_short)

# Concatenate short and long state names with aggregated number of post offices
state_post_offices = pd.concat([state_post_offices_long[['state', 'state_longform', 'num_post_offices']], state_post_offices_long[['state', 'state_longform', 'num_post_offices']]])

# Calculate the centroid of each state and save it to the CSV
centroid_latitudes = []
centroid_longitudes = []

for state_longform in state_post_offices['state_longform']:
    state_data = df_cleaned[df_cleaned['state_longform'] == state_longform]
    centroid_latitude = state_data['latitude'].mean()
    centroid_longitude = state_data['longitude'].mean()
    centroid_latitudes.append(centroid_latitude)
    centroid_longitudes.append(centroid_longitude)

state_post_offices['centroid_latitude'] = centroid_latitudes
state_post_offices['centroid_longitude'] = centroid_longitudes

# Write the aggregated data to a new CSV file
state_post_offices.to_csv("statewise_post_offices.csv", index=False)

print("State-wise post office data saved to statewise_post_offices.csv")


State-wise post office data saved to statewise_post_offices.csv
